# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark==1.3.0
!pip install py4j==0.10.9

# For plotting
!pip install folium
!pip install plotly

In [2]:
!ls /content

ngrok			      spark-3.0.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip  spark-3.0.1-bin-hadoop2.7.tgz.1
sample_data		      spark-3.0.1-bin-hadoop2.7.tgz.2
spark-3.0.1-bin-hadoop2.7


In [3]:
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/libs/libs-kafka_301.zip --directory-prefix=/content/spark-3.0.1-bin-hadoop2.7/jars/
!unzip -n /content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip -d /content/spark-3.0.1-bin-hadoop2.7/jars/
!ls /content/spark-3.0.1-bin-hadoop2.7/jars/*kafka*

Archive:  /content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip
/content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.4.1.jar
/content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip
/content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip.1
/content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip.2
/content/spark-3.0.1-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.12-3.0.1.jar
/content/spark-3.0.1-bin-hadoop2.7/jars/spark-token-provider-kafka-0-10_2.12-3.0.1.jar


Define the environment (Java & Spark homes)

---

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = ""

Starting Spark Session and print the version


---


In [5]:
import findspark
findspark.add_packages(["org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"])
findspark.add_jars(["/content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.0.0.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/lz4-java-1.4.1-jar","/content/spark-3.0.1-bin-hadoop2.7/jars/scala-library-2.11.12.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/slf4j-api-1.7.25.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/snappy-java-1.1.7.1.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-tags_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/unused-1.0.0.jar"])
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

spark.version

'3.0.1'

In [6]:
spark

In [7]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-01-30 12:13:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.196.227.142, 54.145.36.98, 107.21.11.91, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.196.227.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  18.4MB/s    in 0.7s    

2021-01-30 12:13:09 (18.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ngrok                   
http://cab39518e2ee.ngrok.io


# Download Datasets

In [9]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/trades.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/trades.json -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.edges.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.address.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.intermediary.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.officer.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.entity.csv -P /dataset
!ls /dataset

offshore_leaks.edges.csv	       offshore_leaks.nodes.intermediary.csv.1
offshore_leaks.edges.csv.1	       offshore_leaks.nodes.officer.csv
offshore_leaks.nodes.address.csv       offshore_leaks.nodes.officer.csv.1
offshore_leaks.nodes.address.csv.1     trades.csv
offshore_leaks.nodes.entity.csv        trades.csv.1
offshore_leaks.nodes.entity.csv.1      trades.json
offshore_leaks.nodes.intermediary.csv  trades.json.1


# Project 1 - Regulatory Banking Project
---

Input files: /dataset/trades.csv & /dataset/trades.json


Para cada Trade, calcular su valor de ganancia. Porcentaje de ganancia/perdida del cierre respecto de la apertura

In [10]:
df1 = spark.read.format("csv") \
                .option("inferSchema", "true") \
                .option("header", "true") \
                .load("/dataset/trades.csv")

In [11]:
def calc_percentage(open_value, close_value):
  return ((close_value-open_value)/open_value)*100

In [12]:
from pyspark.sql.functions import *

spark.udf.register("udf_calc_percentage", calc_percentage) 

<function __main__.calc_percentage>

In [13]:
df1_trade = df1.withColumn("pnl",round(calc_percentage(col("Open"), col("Close")),2)) \
                .select("Date", "Open", "Close", "pnl") \
                .withColumn("source", lit("trade_1"))

In [14]:
df1_trade.show()

+----------+-------+------+-----+-------+
|      Date|   Open| Close|  pnl| source|
+----------+-------+------+-----+-------+
|2017-12-28| 190.91|189.78|-0.59|trade_1|
|2017-12-27|  190.6|190.19|-0.22|trade_1|
|2017-12-26| 188.53|190.36| 0.97|trade_1|
|2017-12-22|  188.2|188.13|-0.04|trade_1|
|2017-12-21|  187.7|188.08|  0.2|trade_1|
|2017-12-20| 187.14|187.31| 0.09|trade_1|
|2017-12-19|185.235|185.98|  0.4|trade_1|
|2017-12-18| 183.96|184.73| 0.42|trade_1|
|2017-12-15|183.005|182.58|-0.23|trade_1|
|2017-12-14| 183.88|182.13|-0.95|trade_1|
|2017-12-13| 182.01|183.03| 0.56|trade_1|
|2017-12-12| 182.75| 181.8|-0.52|trade_1|
|2017-12-11|  182.9|182.25|-0.36|trade_1|
|2017-12-08|  182.5|183.41|  0.5|trade_1|
|2017-12-07| 180.05| 182.0| 1.08|trade_1|
|2017-12-06| 180.25| 180.8| 0.31|trade_1|
|2017-12-05| 184.79|182.85|-1.05|trade_1|
|2017-12-04| 183.19| 184.9| 0.93|trade_1|
|2017-12-01| 180.32|180.42| 0.06|trade_1|
|2017-11-30| 178.07|179.82| 0.98|trade_1|
+----------+-------+------+-----+-

# Project 2 - Transactions Notifications

*Hint: https://databricks.com/blog/2017/05/08/event-time-aggregation-watermarking-apache-sparks-structured-streaming.html*

In [15]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ec2-34-232-50-77.compute-1.amazonaws.com:9092") \
  .option("subscribe", "transactions") \
  .load()

In [16]:
schema = StructType(
    [
     StructField('Account No', StringType(), True),
     StructField('DATE', StringType(), True),
     StructField('TRANSACTION DETAILS', StringType(), True),
     StructField('CHQ.NO.', StringType(), True),
     StructField('VALUE DATE', StringType(), True),
     StructField(' WITHDRAWAL AMT ', StringType(), True),
     StructField(' DEPOSIT AMT ', StringType(), True),
     StructField('BALANCE AMT', StringType(), True)
    ]
)
df.printSchema()

dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [17]:
dataset.writeStream \
 .outputMode("update") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("transactions") \
 .start()

In [22]:
spark.sql("select * from transactions").show(truncate = False)

KeyboardInterrupt: ignored

Ventana fija de 1 minuto, agrupando por timestamp.
Numero de transacciones en cada minuto

In [20]:
transactions_wd = dataset.groupBy(window(col("timestamp"), "1 minute")).count()

transactions_wd.writeStream \
 .outputMode("update") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("transactions_wd") \
 .start()

In [34]:
spark.sql("select * from transactions_wd group by window, count order by window desc").show(truncate = False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2021-01-30 12:23:00, 2021-01-30 12:24:00]|500  |
|[2021-01-30 12:23:00, 2021-01-30 12:24:00]|400  |
|[2021-01-30 12:23:00, 2021-01-30 12:24:00]|200  |
|[2021-01-30 12:23:00, 2021-01-30 12:24:00]|300  |
|[2021-01-30 12:23:00, 2021-01-30 12:24:00]|100  |
|[2021-01-30 12:22:00, 2021-01-30 12:23:00]|250  |
|[2021-01-30 12:22:00, 2021-01-30 12:23:00]|600  |
|[2021-01-30 12:22:00, 2021-01-30 12:23:00]|350  |
|[2021-01-30 12:22:00, 2021-01-30 12:23:00]|100  |
|[2021-01-30 12:22:00, 2021-01-30 12:23:00]|450  |
|[2021-01-30 12:21:00, 2021-01-30 12:22:00]|300  |
|[2021-01-30 12:21:00, 2021-01-30 12:22:00]|600  |
|[2021-01-30 12:21:00, 2021-01-30 12:22:00]|400  |
|[2021-01-30 12:21:00, 2021-01-30 12:22:00]|50   |
|[2021-01-30 12:21:00, 2021-01-30 12:22:00]|150  |
|[2021-01-30 12:21:00, 2021-01-30 12:22:00]|500  |
|[2021-01-30 12:20:00, 2021-01-

# Project 3 - Panama Papers

Trace "Spring Song International Co., Ltd." entity with Spark SQL using the following dataset</br>
/dataset/offshore_leaks.nodes.entity.csv </br>
/dataset/offshore_leaks.nodes.intermediary.csv </br>
/dataset/offshore_leaks.edges.csv </br>
/dataset/offshore_leaks.nodes.officer.csv